In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [2]:
df = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y , test_size =0.2 , random_state=42)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_trf = scaler.fit_transform(X_train)
X_test_trf = scaler.transform(X_test)

In [8]:
X.shape

(768, 8)

In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

2026-01-19 04:19:05.368981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768796345.648588      18 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768796345.733439      18 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768796346.384981      18 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768796346.385048      18 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768796346.385051      18 computation_placer.cc:177] computation placer alr

In [10]:
model  = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2026-01-19 04:19:22.664245: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [11]:
history = model.fit(X_train_trf, y_train , epochs =100 , batch_size=32, validation_data=(X_test_trf,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4630 - loss: 0.7054 - val_accuracy: 0.5714 - val_loss: 0.6865
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6407 - loss: 0.6563 - val_accuracy: 0.6948 - val_loss: 0.6430
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7121 - loss: 0.6264 - val_accuracy: 0.7143 - val_loss: 0.6087
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7393 - loss: 0.5786 - val_accuracy: 0.7143 - val_loss: 0.5842
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7431 - loss: 0.5595 - val_accuracy: 0.7273 - val_loss: 0.5637
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7434 - loss: 0.5268 - val_accuracy: 0.7143 - val_loss: 0.5507
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7590 - loss: 0.5167 - val_accuracy: 0.7143 - val_loss: 0.5424
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7678 - loss: 0.4980 - val_accuracy: 0.7208 - 

# How to select best optimizer

In [12]:
import keras_tuner as kt
#1.write a model building Function
#2.Choose a tuner
#3.Run the search
#4.Get the best model  


In [13]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer = hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [14]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials = 5,
                       directory ='mydir',
                       project_name='optimizers_tunning')

In [15]:
tuner.search(X_train_trf,y_train,epochs=5,validation_data=(X_test_trf,y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.6623376607894897

Best val_accuracy So Far: 0.7467532753944397
Total elapsed time: 00h 00m 10s


In [16]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [17]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [18]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.fit(X_train_trf,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test_trf,y_test))
#initial_epoch=6 , becoz earlier model has run 5 epochs already

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7730 - loss: 0.5339 - val_accuracy: 0.7532 - val_loss: 0.5328
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7492 - loss: 0.5170 - val_accuracy: 0.7597 - val_loss: 0.5229
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7392 - loss: 0.4887 - val_accuracy: 0.7468 - val_loss: 0.5138
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7299 - loss: 0.5197 - val_accuracy: 0.7597 - val_loss: 0.5079
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7645 - loss: 0.4731 - val_accuracy: 0.7662 - val_loss: 0.5034
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7651 - loss: 0.4666 - val_accuracy: 0.7532 - val_loss: 0.5003
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7753 - loss: 0.4522 - val_accuracy: 0.7532 - val_loss: 0.4981
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7691 - loss: 0.4724 - val_accuracy: 0.75

# Number of Neurons in a particular layer

In [20]:
def build_model(hp):
    model=Sequential()
    units = hp.Int(name='units',min_value=8,max_value=128,step=8)
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [21]:
tuner = kt.RandomSearch(build_model ,
                        objective='val_accuracy',
                       max_trials=5,
                       directory = 'mydir',
                       project_name ='Neuron_num_tunning') #try 5 diff hyperpara combinations

In [22]:
tuner.search(X_train_trf,y_train ,epochs=5,validation_data=(X_test_trf,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 11s


In [23]:
tuner.get_best_hyperparameters()[0].values

{'units': 80}

In [24]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(X_train_trf,y_train,epochs=100,initial_epoch=6,validation_data=(X_test_trf,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7616 - loss: 0.5139 - val_accuracy: 0.8117 - val_loss: 0.4885
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7661 - loss: 0.4889 - val_accuracy: 0.8117 - val_loss: 0.4835
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7673 - loss: 0.4726 - val_accuracy: 0.8247 - val_loss: 0.4792
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7764 - loss: 0.4759 - val_accuracy: 0.7987 - val_loss: 0.4759
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7716 - loss: 0.4815 - val_accuracy: 0.7987 - val_loss: 0.4744
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7688 - loss: 0.4551 - val_accuracy: 0.7987 - val_loss: 0.4728
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7756 - loss: 0.4492 - val_accuracy: 0.7987 - val_loss: 0.4723
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7762 - loss: 0.4638 - val_accuracy: 0.78

# No of Layers

In [25]:
def build_model(hp):
    model=  Sequential()
    model.add(Dense(72,activation='relu',input_dim=8))
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [26]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials = 3,
                       directory = 'mydir',
                       project_name='no_layers')

In [27]:
tuner.search(X_train_trf,y_train,validation_data=(X_test_trf,y_test),epochs=5)

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 10s


In [28]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10}

In [29]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [30]:
model.fit(X_train_trf,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.7832 - loss: 0.4764 - val_accuracy: 0.3571 - val_loss: 31.0500
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7883 - loss: 0.4690 - val_accuracy: 0.3571 - val_loss: 60.4216
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8204 - loss: 0.4070 - val_accuracy: 0.3442 - val_loss: 27.1126
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8094 - loss: 0.4212 - val_accuracy: 0.3506 - val_loss: 51.1765
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7955 - loss: 0.4306 - val_accuracy: 0.3571 - val_loss: 46.3798
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8519 - loss: 0.3527 - val_accuracy: 0.3571 - val_loss: 55.6981
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8416 - loss: 0.3917 - val_accuracy: 0.3506 - val_loss: 57.0390
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8354 - loss: 0.4048 - val_accurac

# All Together

In [31]:
from keras.layers import Dropout

In [32]:
def build_model(hp):
    model = Sequential()

    num_layers = hp.Int('num_layers', min_value=1, max_value=10)

    for i in range(num_layers):
        units = hp.Int(f'units_{i}', min_value=8, max_value=128, step=8)
        activation = hp.Choice(f'activation_{i}', ['relu', 'tanh', 'sigmoid'])
        dropout_rate = hp.Choice(f'dropout_{i}', [0.1,0.2,0.3,0.4,0.5])

        if i == 0:
            model.add(Dense(units, activation=activation, input_shape=(8,)))
        else:
            model.add(Dense(units, activation=activation))

        model.add(Dropout(dropout_rate))

    # OUTPUT LAYER (once)
    model.add(Dense(1, activation='sigmoid'))

    # OPTIMIZER
    optimizer = hp.Choice(
        'optimizer',
        ['adam', 'rmsprop', 'sgd', 'nadam', 'adadelta']
    )

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [33]:
tuner = kt.RandomSearch(build_model,
                       max_trials = 3,
                       objective = 'val_accuracy',
                       directory = 'mydir',
                       project_name ='all_new')

In [34]:
tuner.search(X_train_trf,y_train,epochs=5,validation_data=(X_test_trf,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.3571428656578064

Best val_accuracy So Far: 0.6753246784210205
Total elapsed time: 00h 00m 11s


In [35]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units_0': 88,
 'activation_0': 'relu',
 'dropout_0': 0.3,
 'optimizer': 'sgd',
 'units_1': 96,
 'activation_1': 'tanh',
 'dropout_1': 0.1,
 'units_2': 8,
 'activation_2': 'relu',
 'dropout_2': 0.1,
 'units_3': 8,
 'activation_3': 'relu',
 'dropout_3': 0.1,
 'units_4': 8,
 'activation_4': 'relu',
 'dropout_4': 0.1,
 'units_5': 8,
 'activation_5': 'relu',
 'dropout_5': 0.1}

In [36]:
model  = tuner.get_best_models(num_models = 1)[0]

In [37]:
model.fit(X_train_trf,y_train, epochs=100 , initial_epoch= 6,validation_data=(X_test_trf,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6236 - loss: 0.6791 - val_accuracy: 0.6623 - val_loss: 0.6710
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6535 - loss: 0.6745 - val_accuracy: 0.6558 - val_loss: 0.6673
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6558 - loss: 0.6639 - val_accuracy: 0.6364 - val_loss: 0.6635
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6587 - loss: 0.6637 - val_accuracy: 0.6364 - val_loss: 0.6604
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6306 - loss: 0.6666 - val_accuracy: 0.6364 - val_loss: 0.6581
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6864 - loss: 0.6506 - val_accuracy: 0.6429 - val_loss: 0.6553
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6717 - loss: 0.6510 - val_accuracy: 0.6429 - val_loss: 0.6528
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6628 - loss: 0.6489 - val_accuracy: 0.64